In [1]:
from captcha.image import ImageCaptcha  # pip install captcha
import numpy as np
from PIL import Image
import random
import matplotlib.pyplot as plt
import os
from random import choice
import tensorflow as tf
%load_ext autoreload
%autoreload 2

E:\06_software\Anoconda\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
import verification_decoder as vd

In [4]:
# tf.set_random_seed(1)
image = tf.placeholder(tf.float32, [32, 60, 180, 1])  # 定义图片的大小
# 定义每个预测值得维度
labels = dict(
    digit1=tf.placeholder(tf.float32, [None, 36]),
    digit2=tf.placeholder(tf.float32, [None, 36]),
    digit3=tf.placeholder(tf.float32, [None, 36]),
    digit4=tf.placeholder(tf.float32, [None, 36])
)

training_options = dict(
    drop_rate=0.9,
    learning_rate=1e-3,  # 学习率
    decay_steps=10000,  # 多少步降低学习率
    decay_rate=1,  # 每次降低 1 - decay_rate
    batch_size=32,  # 每次训练多少张图片
    show_loss=20,  # 貌似没用到
    total_episode=50,  # 总训练回合
    show_test=1000,  # 多少步展示一下测试数据的预测率以及预测值、真实值
    output_board=True,  # 是否输出到tensorboard
    logs_step=10000,  # 多少步往tensorboard里写入 同时存放model
    save_step=10000,
    log_path=r'E:\07_data\00_practise\verification_code\logs\\',  # tensorboard的log文件存放在哪里
    model_path=r'E:\07_data\00_practise\verification_code\net_model\\',  # model保存在那个文件夹
    model_name='model.ckpt'  # model文件名
)

In [5]:
model = vd.model.Model()  # 初始化model类
config = tf.ConfigProto()
config.gpu_options.allow_growth=False

In [7]:
with tf.Session(config=config) as sess:
    if not os.path.exists(training_options['model_path']):
        os.mkdir(training_options['model_path'])
    # 定义神经网络
    net, train = model.build_network(training_options=training_options, image=image,
                                     drop_rate=training_options['drop_rate'], labels=labels)
    # 如果写入tensorboard
    if training_options['output_board']:
        merged = tf.summary.merge_all()  # tensorflow >= 0.12
        writer = tf.summary.FileWriter(training_options['log_path'], sess.graph)  # tensorflow >=0.12
    # 获得saver对象，可以保存model以及读取model
    saver = tf.train.Saver()
    # 如果已经存在model副本就直接读取，否则就初始化神经网络参数
    if os.path.exists(training_options['model_path'] + '/checkpoint'):
        saver.restore(sess, training_options['model_path'] + training_options['model_name'])
    else:
        sess.run(tf.global_variables_initializer())
    # 初始化 imageUtils类，获得所有训练，测试数据
    imageUtils = vd.image_utils.ImageUtils()
    train_data = imageUtils.train_data
    train_label = imageUtils.train_label
    test_data = imageUtils.test_data
    test_label = imageUtils.test_label
    # 开始训练啦
    for episode in range(training_options['total_episode']):
        # 随机获得定义好数量的训练数据
        sample_datas, sample_labels = imageUtils.sample(len(train_data), training_options['batch_size'],
                                                        train_data, train_label)
        # 训练神经网络
        global_step, _, loss = sess.run(
            [train['global_step'], train['train'], train['loss']],
            feed_dict={image: sample_datas, labels['digit1']: sample_labels[:, 0],
                       labels['digit2']: sample_labels[:, 1],
                       labels['digit3']: sample_labels[:, 2],
                       labels['digit4']: sample_labels[:, 3]})
        # 打印 [总共训练了多少回合， loss值]
        print('total episode: {0:}\t\tloss: {1:.4f}'.format(global_step, loss))
        # 指定的回合数时保存tensorboard log文件，保存model（神经网络参数）
        if episode % training_options['logs_step'] == 0 and training_options['output_board']:
            result = sess.run(merged, feed_dict={image: sample_datas, labels['digit1']: sample_labels[:, 0],
                                                 labels['digit2']: sample_labels[:, 1],
                                                 labels['digit3']: sample_labels[:, 2],
                                                 labels['digit4']: sample_labels[:, 3]})
            writer.add_summary(result, global_step)
        if episode != 0 and episode % training_options['save_step'] == 0:
            saver.save(sess, training_options['model_path'] + training_options['model_name'])
        # 指定回合数时 打印预测值、真实值、正确率
        if episode % training_options['show_test'] == 0:
            t_sample_datas, t_sample_labels = imageUtils.sample(len(test_data), 10, test_data, test_label)
            result = sess.run([net['digit1'], net['digit2'], net['digit3'], net['digit4']],
                              feed_dict={image: t_sample_datas})
            result = code_utils.batch_out_transition(result)
            predicted = [result[0][index] + result[1][index] + result[2][index] + result[3][index]
                         for index in range(len(result[0]))]
            label = code_utils.batch_out_transition(t_sample_labels)
            four_right_count = np.count_nonzero([predicted[index] == label[index] for index in range(len(predicted))])
            one_right_count = np.count_nonzero(
                [predicted[index][s_index] == label[index][s_index] for index in range(len(predicted)) for s_index in
                 range(len(predicted[index]))])
            print('predicted:\t{} \nlabel:\t\t{}'.format(predicted,
                                                         label))
            print('4 match: {0:.2f}%\t\t1 match: {1:.2f}%'.format(four_right_count / 10 * 100,
                                                                  one_right_count / 40 * 100))

ValueError: Dimension size must be evenly divisible by 2304 but is 376832 for 'CNN/Reshape' (op: 'Reshape') with input shapes: [32,8,23,64], [2] and with input tensors computed as partial shapes: input[1] = [?,2304].

In [8]:
imageUtils = vd.image_utils.ImageUtils()
train_data = imageUtils.train_data
train_label = imageUtils.train_label
test_data = imageUtils.test_data
test_label = imageUtils.test_label

In [9]:
sample_datas, sample_labels = imageUtils.sample(len(train_data), training_options['batch_size'],
                                                        train_data, train_label)

In [22]:
with tf.Session(config=config) as sess:
    global_step, _, loss = sess.run(
        [train['global_step'], train['train'], train['loss']],
        feed_dict={image: sample_datas, labels['digit1']: sample_labels[:, 0],
                   labels['digit2']: sample_labels[:, 1],
                   labels['digit3']: sample_labels[:, 2],
                   labels['digit4']: sample_labels[:, 3]})

(32, 4, 36)

In [48]:
sample_index = np.random.choice(len(train_data), training_options['batch_size'])

In [10]:
image = tf.placeholder(tf.float32, [32, 60, 180, 1])

In [11]:
net, train = model.build_network(training_options=training_options, image=image,
                                 drop_rate=training_options['drop_rate'], labels=labels)

ValueError: Variable CNN/hidden1/conv2d/kernel already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "E:\06_software\Anoconda\lib\site-packages\tensorflow\python\framework\ops.py", line 1768, in __init__
    self._traceback = tf_stack.extract_stack()
  File "E:\06_software\Anoconda\lib\site-packages\tensorflow\python\framework\ops.py", line 3272, in create_op
    op_def=op_def)
  File "E:\06_software\Anoconda\lib\site-packages\tensorflow\python\util\deprecation.py", line 488, in new_func
    return func(*args, **kwargs)


In [13]:
with tf.variable_scope('CNN') as cnn:
    # 卷积卷积卷积卷积
    with tf.variable_scope('hidden1'):
        hidden1 = cnn(image, 32, kernel_size=[3, 3])
    with tf.variable_scope('hidden2'):
        hidden2 = cnn(hidden1, 64, kernel_size=[3, 3])
    with tf.variable_scope('hidden3'):
        hidden3 = cnn(hidden2, 64, kernel_size=[3, 3])

TypeError: 'VariableScope' object is not callable

In [38]:
a[1].split('\\')[-1].split('.')[0].split('_')[0]

'c05g'

In [36]:
import glob
a = glob.glob(r'E:\07_data\00_practise\verification_code\train/*.jpeg')

In [34]:
a

['E:\\07_data\\00_practise\\verification_code\\train\\c05c.jpeg',
 'E:\\07_data\\00_practise\\verification_code\\train\\c05g.jpeg',
 'E:\\07_data\\00_practise\\verification_code\\train\\c05i.jpeg',
 'E:\\07_data\\00_practise\\verification_code\\train\\c05k.jpeg',
 'E:\\07_data\\00_practise\\verification_code\\train\\c05m.jpeg',
 'E:\\07_data\\00_practise\\verification_code\\train\\c05n.jpeg',
 'E:\\07_data\\00_practise\\verification_code\\train\\c05s.jpeg',
 'E:\\07_data\\00_practise\\verification_code\\train\\c05w.jpeg',
 'E:\\07_data\\00_practise\\verification_code\\train\\c05y.jpeg',
 'E:\\07_data\\00_practise\\verification_code\\train\\c06a.jpeg',
 'E:\\07_data\\00_practise\\verification_code\\train\\c06r.jpeg',
 'E:\\07_data\\00_practise\\verification_code\\train\\c06x.jpeg',
 'E:\\07_data\\00_practise\\verification_code\\train\\c07a.jpeg',
 'E:\\07_data\\00_practise\\verification_code\\train\\c07k.jpeg',
 'E:\\07_data\\00_practise\\verification_code\\train\\c07m.jpeg',
 'E:\\07_d

# 随机读取训练集

In [2]:
# 验证码中的字符, 就不用汉字了
number = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
alphabet = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u',
            'v', 'w', 'x', 'y', 'z']
# ALPHABET = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U',
#             'V', 'W', 'X', 'Y', 'Z']

root_dir = r"E:\07_data\00_practise\verification_code\train"

# 验证码一般都无视大小写；验证码长度4个字符
def random_captcha_text(char_set=number, captcha_size=4):
    captcha_text = []
    for i in range(captcha_size):
        c = random.choice(char_set)
        captcha_text.append(c)
    return captcha_text


# 生成字符对应的验证码
def gen_captcha_text_and_image():
    image = ImageCaptcha()

    captcha_text = random_captcha_text()
    captcha_text = ''.join(captcha_text)

    captcha = image.generate(captcha_text)
    # image.write(captcha_text, captcha_text + '.jpg')  # 写到文件

    captcha_image = Image.open(captcha)
    captcha_image = np.array(captcha_image)
    return captcha_text, captcha_image



def gen_list():
    img_list = []
    for parent, dirnames, filenames in os.walk(root_dir):  # 三个参数：分别返回1.父目录 2.所有文件夹名字（不含路径） 3.所有文件名字
        for filename in filenames:  # 输出文件信息
            img_list.append(filename.replace(".jpg", ""))
            # print("parent is:" + parent)
            # print("filename is:" + filename)
            # print("the full name of the file is:" + os.path.join(parent, filename))  # 输出文件路径信息
    return img_list

img_list = gen_list()

def gen_captcha_text_and_image_new():
    img = choice(img_list)
    captcha_image = Image.open(root_dir + "\\" + img + ".jpg")
    # captcha_image = captcha_image.resize((90, 30))
    captcha_image = np.array(captcha_image)
    return img, captcha_image

# 读取测试集

In [3]:
root_dir = r"E:\07_data\00_practise\verification_code\test"
img_list = []

def gen_list():

    for parent, dirnames, filenames in os.walk(root_dir):  # 三个参数：分别返回1.父目录 2.所有文件夹名字（不含路径） 3.所有文件名字
        for filename in filenames:  # 输出文件信息
            img_list.append(filename.replace(".jpg", ""))
            # print("parent is:" + parent)
            # print("filename is:" + filename)
            # print("the full name of the file is:" + os.path.join(parent, filename))  # 输出文件路径信息
    return img_list

img_list = gen_list()
def get_test_captcha_text_and_image(i=None):
    img = img_list[i]
    captcha_image = Image.open(root_dir + "\\" + img + ".jpg")
    # captcha_image = captcha_image.resize((160, 60))
    captcha_image = np.array(captcha_image)
    return img, captcha_image

def get_test_sets_length():
    return len(img_list)

# 设计神经网络

In [4]:
text, image = gen_captcha_text_and_image_new()
print("验证码图像channel:", image.shape)  # (60, 160, 3)
# 图像大小
IMAGE_HEIGHT = image.shape[0]
IMAGE_WIDTH = image.shape[1]
image_shape = image.shape
MAX_CAPTCHA = len(text)
print("验证码文本最长字符数", MAX_CAPTCHA)  # 验证码最长4字符; 我全部固定为4,可以不固定. 如果验证码长度小于4，用'_'补齐

# 把彩色图像转为灰度图像（色彩对识别验证码没有什么用）
# 度化是将三分量转化成一样数值的过程
def convert2gray(img):
    if len(img.shape) > 2:
        gray = np.mean(img, -1)
        # 上面的转法较快，正规转法如下
        # r, g, b = img[:,:,0], img[:,:,1], img[:,:,2]
        # gray = 0.2989 * r + 0.5870 * g + 0.1140 * b
        # int gray = (int) (0.3 * r + 0.59 * g + 0.11 * b);
        return gray
    else:
        return img


""" 
cnn在图像大小是2的倍数时性能最高, 如果你用的图像大小不是2的倍数，可以在图像边缘补无用像素。 
np.pad(image,((2,3),(2,2)), 'constant', constant_values=(255,))  # 在图像上补2行，下补3行，左补2行，右补2行 
"""


char_set = number + alphabet   # 如果验证码长度小于4, '_'用来补齐
CHAR_SET_LEN = len(char_set)

# 文本转向量
def text2vec(text):
    text_len = len(text)
    if text_len > MAX_CAPTCHA:
        raise ValueError('验证码最长4个字符')

    vector = np.zeros(MAX_CAPTCHA * CHAR_SET_LEN)

    def char2pos(c):
        try:
            if ord(c) <= ord('9'):
                k = ord(c)-ord('0')
            else:
                k = ord(c)-ord('a') + 10
        except:
            raise ValueError('No Map')
        return k

    for i, c in enumerate(text):
        idx = i * CHAR_SET_LEN + char2pos(c)
        vector[idx] = 1
    return vector


# 向量转回文本
def vec2text(vec):
    char_pos = vec.nonzero()[0]
    text = []
    for i, c in enumerate(char_pos):
        char_at_pos = i  # c/63
        char_idx = c % CHAR_SET_LEN
        if char_idx < 10:
            char_code = char_idx + ord('0')
        elif char_idx < 36:
            char_code = char_idx - 10 + ord('A')
        elif char_idx < 62:
            char_code = char_idx - 36 + ord('a')
        elif char_idx == 62:
            char_code = ord('_')
        else:
            raise ValueError('error')
        text.append(chr(char_code))
    return "".join(text)


# 生成一个训练batch
def get_next_batch(batch_size=128):
    batch_x = np.zeros([batch_size, IMAGE_HEIGHT * IMAGE_WIDTH])
    batch_y = np.zeros([batch_size, MAX_CAPTCHA * CHAR_SET_LEN])

    # 有时生成图像大小不是(60, 160, 3)
    def wrap_gen_captcha_text_and_image():
        while True:
            text, image = gen_captcha_text_and_image_new()
            if image.shape == image_shape:
                return text, image

    for i in range(batch_size):
        text, image = wrap_gen_captcha_text_and_image()
        image = convert2gray(image)


        batch_x[i, :] = image.flatten() / 255  # (image.flatten()-128)/128  mean为0
        batch_y[i, :] = text2vec(text)

    return batch_x, batch_y



验证码图像channel: (60, 180, 3)
验证码文本最长字符数 4


In [5]:
X = tf.placeholder(tf.float32, [None, IMAGE_HEIGHT * IMAGE_WIDTH])
Y = tf.placeholder(tf.float32, [None, MAX_CAPTCHA * CHAR_SET_LEN])
keep_prob = tf.placeholder(tf.float32)  # dropout

In [6]:
# 定义CNN
def crack_captcha_cnn(w_alpha=0.01, b_alpha=0.1):
    x = tf.reshape(X, shape=[-1, IMAGE_HEIGHT, IMAGE_WIDTH, 1])

    # w_c1_alpha = np.sqrt(2.0/(IMAGE_HEIGHT*IMAGE_WIDTH)) #
    # w_c2_alpha = np.sqrt(2.0/(3*3*32))
    # w_c3_alpha = np.sqrt(2.0/(3*3*64))
    # w_d1_alpha = np.sqrt(2.0/(8*32*64))
    # out_alpha = np.sqrt(2.0/1024)

    # 定义三层的卷积神经网络

    # 定义第一层的卷积神经网络
    # 定义第一层权重
    w_c1 = tf.Variable(w_alpha * tf.random_normal([3, 3, 1, 32]))
    # 定义第一层的偏置
    b_c1 = tf.Variable(b_alpha * tf.random_normal([32]))
    # 定义第一层的激励函数
    conv1 = tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(x, w_c1, strides=[1, 1, 1, 1], padding='SAME'), b_c1))
    # conv1 为输入  ksize 表示使用2*2池化，即将2*2的色块转化成1*1的色块
    conv1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    # dropout防止过拟合。
    conv1 = tf.nn.dropout(conv1, keep_prob)

    # 定义第二层的卷积神经网络
    w_c2 = tf.Variable(w_alpha * tf.random_normal([3, 3, 32, 64]))
    b_c2 = tf.Variable(b_alpha * tf.random_normal([64]))
    conv2 = tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(conv1, w_c2, strides=[1, 1, 1, 1], padding='SAME'), b_c2))
    conv2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    conv2 = tf.nn.dropout(conv2, keep_prob)

    # 定义第三层的卷积神经网络
    w_c3 = tf.Variable(w_alpha * tf.random_normal([3, 3, 64, 64]))
    b_c3 = tf.Variable(b_alpha * tf.random_normal([64]))
    conv3 = tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(conv2, w_c3, strides=[1, 1, 1, 1], padding='SAME'), b_c3))
    conv3 = tf.nn.max_pool(conv3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    conv3 = tf.nn.dropout(conv3, keep_prob)

    # Fully connected layer
    # 随机生成权重
    w_d = tf.Variable(w_alpha * tf.random_normal([11776, 2048]))
    # 随机生成偏置
    b_d = tf.Variable(b_alpha * tf.random_normal([2048]))
    dense = tf.reshape(conv3, [-1, w_d.get_shape().as_list()[0]])
    dense = tf.nn.relu(tf.add(tf.matmul(dense, w_d), b_d))
    dense = tf.nn.dropout(dense, keep_prob)

    w_out = tf.Variable(w_alpha * tf.random_normal([2048, MAX_CAPTCHA * CHAR_SET_LEN]))
    b_out = tf.Variable(b_alpha * tf.random_normal([MAX_CAPTCHA * CHAR_SET_LEN]))
    out = tf.add(tf.matmul(dense, w_out), b_out)
    # out = tf.nn.softmax(out)
    return out


# 训练
def train_crack_captcha_cnn():
    # X = tf.placeholder(tf.float32, [None, IMAGE_HEIGHT * IMAGE_WIDTH])
    # Y = tf.placeholder(tf.float32, [None, MAX_CAPTCHA * CHAR_SET_LEN])
    # keep_prob = tf.placeholder(tf.float32)  # dropout
    output = crack_captcha_cnn()
    # loss
    # loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(output, Y))
    loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=output, labels=Y))
    # 最后一层用来分类的softmax和sigmoid有什么不同？
    # optimizer 为了加快训练 learning_rate应该开始大，然后慢慢衰
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)

    predict = tf.reshape(output, [-1, MAX_CAPTCHA, CHAR_SET_LEN])
    max_idx_p = tf.argmax(predict, 2)
    max_idx_l = tf.argmax(tf.reshape(Y, [-1, MAX_CAPTCHA, CHAR_SET_LEN]), 2)
    correct_pred = tf.equal(max_idx_p, max_idx_l)
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

    saver = tf.train.Saver()
    with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())

            step = 0
            while True:
                print(step)
                batch_x, batch_y = get_next_batch(64)
                _, loss_ = sess.run([optimizer, loss], feed_dict={X: batch_x, Y: batch_y, keep_prob: 0.75})
                print(step, loss_)

                # 每100 step计算一次准确率
                if step % 100 == 0:
                    batch_x_test, batch_y_test = get_next_batch(1600)
                    acc = sess.run(accuracy, feed_dict={X: batch_x_test, Y: batch_y_test, keep_prob: 1.})
                    print(step, acc)
                    # 如果准确率大于50%,保存模型,完成训练
                    if (acc > 0.9999999999) & (step > 20000):
                        saver.save(sess, "./crack_capcha.model", global_step=step)
                        break
                step += 1

In [ ]:
## 训练(如果要训练则去掉下面一行的注释)
train_crack_captcha_cnn()

0
0 0.6846864
0 

In [ ]:


## 训练(如果要训练则去掉下面一行的注释)
train_crack_captcha_cnn()



def crack_captcha():
    output = crack_captcha_cnn()

    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, tf.train.latest_checkpoint('.'))

        predict = tf.argmax(tf.reshape(output, [-1, MAX_CAPTCHA, CHAR_SET_LEN]), 2)
        count = 0


        # print("正确率:" + str(count) + "/1324")
        for i in range(get_test_sets_length()):
            text, image = get_test_captcha_text_and_image(i)
            image = convert2gray(image)
            captcha_image = image.flatten() / 255
            text_list = sess.run(predict, feed_dict={X: [captcha_image], keep_prob: 1})
            predict_text = text_list[0].tolist()
            # predict_text = str(predict_text)
            # predict_text = predict_text.replace("[", "").replace("]", "").replace(",", "").replace(" ","")
            tmp = ''
            for char_idx in predict_text:
                if char_idx < 10:
                    char_code = char_idx + ord('0')
                elif char_idx < 36:
                    char_code = char_idx - 10 + ord('a')
                tmp = tmp + chr(char_code)
            predict_text = tmp

            if text == predict_text:
                count += 1
                check_result = ",预测结果正确"
            else:
                check_result = ",预测结果不正确"
            print(str(i) + ':' + predict_text + check_result)

        print("正确率:" + str(count) + "/" + str(get_test_sets_length()))